In [1]:
from konlpy.tag import Okt
from konlpy.tag import Kkma
from kiwipiepy import Kiwi
import pandas as pd
import numpy as np
import koreanize_matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from PIL import Image 
import re
from collections import Counter
from emoji import core

In [2]:
df = pd.read_csv(
    "../data/saramin_merge_sections_split_v3.csv"
)

df.head()

,keyword,title,company,location,career,education,work_type,deadline,link,rec_idx,keyword_text,crawled_at,corp_domain,totaltext,responsibilities,qualifications,preferred_quali,hiring_process,benefits,salary
0,AI 에이전트,[AI기반 특허 데이터분석 플랫폼 기업] 프론트엔드 개발자,에이피써치,서울 강남구,경력 7~15년,대졸↑,정규직,채용시,https://www.saramin.co.kr/zf_user/jobs/relay/v...,52982351,프론트엔드 헤드헌팅,2026-02-02 12:27:26,23명 (2017년 기준),[AI기반 특허 데이터분석 플랫폼 기업] 프론트엔드 개발자\n프론트엔드 개발자 (1...,회사\nAI 기반 특허 데이터분석 플랫폼 스타트업\n2025 아기 유니콘 기업 선정...,Frontend 개발 경험을 7년 이상 보유하고 있으신 분\nJavaScript와 ...,TypeScript에 대한 경험과 이해가 깊은 분\n대규모 프로젝트나 복잡한 업무를...,E-mail : wonmi.ahn@apsearch.co.kr\n===========...,고용형태 : 정규직,"이력서 제출시 이직사유(퇴직사유), 최종연봉 및 희망연봉 필수 기재요망(기본급/성과..."
1,AI 에이전트,[자율주행] AI Engineer / 3D Auto-labeling,(주)비저너리,서울 강남구,경력무관,석사↑,정규직,채용시,https://www.saramin.co.kr/zf_user/jobs/relay/v...,52981267,AI(인공지능) AI(인공지능),2026-02-02 12:20:37,응용 소프트웨어 개발 및 공급업,[자율주행] AI Engineer | 3D Auto-labeling / Spatia...,3D Perception 태스크 자동 가공 모델 개발 및 학습 파이프라인 구축\nD...,관련 전공 석사 이상 학력 혹은 그에 준하는 경력 및 실력\n딥러닝 프로젝트를 학습...,코드 리뷰를 통해 더 좋은 구조 논의\n주도적으로 문제를 정의하고 해결하는 분\n시...,Spatial Data Processing\n합류 과정\n서류 검토\n코딩테스트나 ...,주 3일 출근 (서울 선릉) + 주 2일 재택\n00~16:00\n장비 및 인프라\...,지원서 접수 → (선택) 커피챗 → 기술 인터뷰 → 필요 시 추가 인터뷰(컬처/오프...
2,AI 에이전트,[로봇 솔루션기업] 모바일로봇 교통 관제 시스템 설계 및 개발자,(주)맨토스파워,서울 마포구,경력2년↑,대졸↑,정규직,~ 02/17(화),https://www.saramin.co.kr/zf_user/jobs/relay/v...,52980063,AI(인공지능) AI(인공지능) 로봇설계 알고리즘 MAPF,2026-02-02 12:11:50,임시 및 일용 인력 공급업,[코스닥상장/로봇 소프트웨어 개발사] \n모바일로봇 교통 관제 시스템 설계 및 개발...,"멀티 로봇 경로 계획(Multi-Agent Path Finding, MAPF) 알고...","해당 분야 석사(신입) 또는 관련 분야 경력 2년 이상\n성실하고, 다양한 이해관계...","로봇, AI 관련 업무 유경험자\nMulti-Agent Path Finding(MA...",e-mail : tender@mantp.co.kr / tender3599@naver...,NaN,NaN
3,AI 에이전트,Microsoft 365 Copilot 및 Agent 개발 컨설턴트,(주)무늬랩스,서울 송파구,경력2년↑,초대졸↑,정규직,~ 03/04(수),https://www.saramin.co.kr/zf_user/jobs/relay/v...,52979929,기술지원 백엔드/서버개발 AI(인공지능) 데이터분석가 데이터엔지니어,2026-02-02 12:22:23,경영 컨설팅업,Microsoft 365 Copilot 및 Agent 개발 컨설턴트\nM365 Co...,고객 맞춤형 M365 Copilot & Agent 활용 방안 제안 및 Copilot...,"경력 2년 이상\n대학졸업(2,3년)이상\nCopilot Agent 기획/개발/구현...","Copilot Studio (Builder), Power Automate, Powe...",서류전형 → 1차면접 (부문 리더) → 2차면접 (대표이사) → 최종합격,M365/Copilot/Agent 도입 변화관리 차원의 사용자 교육 및 가이드 개발...,개인성장지원비 지급 (인당 연 100만원\n급여: 협의 후 결정
4,AI 에이전트,35. AI Platform Engineer [대기업 게임사],베스트네트워크㈜,서울 강남구,경력2년↑,대졸↑,정규직,~ 02/17(화),https://www.saramin.co.kr/zf_user/jobs/relay/v...,52979323,AI(인공지능) 기술연구,2026-02-02 12:22:02,경영 컨설팅업,35. AI Platform Engineer [대기업 게임사]\nAI Platfor...,사내 유저들의 워크플로우를 분석·개선하여 공통 문제를 해결하고 획기적인 생산성 개선...,3년 이상 경력\n소프트웨어 엔지니어링: FE → BE → 인프라 배포/모니터링까지...,NaN,제출서류: 사람인 온라인 이력서\n접수방법: 사람인 입사지원\n전형절차: 서류전형 ...,고용형태 : 정규직,급여: 면접 후 결정


In [3]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   keyword           478 non-null    str  
 1   title             478 non-null    str  
 2   company           478 non-null    str  
 3   location          478 non-null    str  
 4   career            478 non-null    str  
 5   education         478 non-null    str  
 6   work_type         478 non-null    str  
 7   deadline          478 non-null    str  
 8   link              478 non-null    str  
 9   rec_idx           478 non-null    int64
 10  keyword_text      478 non-null    str  
 11  crawled_at        478 non-null    str  
 12  corp_domain       441 non-null    str  
 13  totaltext         478 non-null    str  
 14  responsibilities  478 non-null    str  
 15  qualifications    439 non-null    str  
 16  preferred_quali   392 non-null    str  
 17  hiring_process    451 non-null    str  
 18  b

In [4]:
#주요업무 토크나이징

kiwi = Kiwi()
index = 0
df["respons_text"] = None
for sent in df["responsibilities"]:
    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)
    result = kiwi.tokenize(clean_sent)
    sub_result = []    
    for x in result:
        word = x.form
        pos = x.tag
        if pos[0] in ["J", "E", "X"] or len(word) <= 1:
            continue
        sub_result.append(word)
    print(f"index: {index}, sub_list: {sub_result}")
    if len(sub_result) > 0:
        df.at[index, "respons_text"] = sub_result

    index += 1


df["respons_text"]


index: 0, sub_list: ['회사', 'AI', '기반', '특허', '데이터', '분석', '플랫폼', '스타트업', '아기', '유니콘', '기업', '선정', '사용자', '경험', '극대', '위하', '창의', '실용', '기능', '설계', '구현', '결과', '도달', '자연어', '검색', '시각', '실시간', '반응', '서비스', 'AI', '에이전트', '통하', '자동', '분류', '문서', '생성', '설계', '구현', '신규', '서비스', '기획', '개발', '최신', '기술', '적용', '코드', '개선', 'React', 'Nextjs', '기반', '컴포넌트', '개발']
index: 1, sub_list: ['Perception', '태스크', '자동', '가공', '모델', '개발', '학습', '파이프라인', '구축', 'Detection', 'Tracking', 'Depth', 'Prediction', 'Segmentation', 'Occupancy', 'HD', 'Map', '자동', '가공', '품질', '제품', '수준', '만들', '위하', '평가', '체계', '설계', 'Test', 'Set', 'Metric', 'Regression', 'Test', '실패', '케이스', '정의', 'Edge', 'case', '자동', '파악', '개선', '루프', '구축', '신뢰도', '불확실', '기반', '데이터', '검수', '우선순위', '설계', '피드백', '반영', '통하', '모델', '지속', '개선', '고객', '요구', '포맷', 'Schema', '맞추', '결과물', '표준', '재현', '가능', '실험', '체계', '구축', '데이터', '모델', '버저닝', '함께', '시간', 'TB', '이상', '자율', '주행', '데이터', '직접', '운영', '경험', '가지', '공동', '창업자', '함께', 'KAIST', '서울대', '박사', 'ICCV', 

0      [회사, AI, 기반, 특허, 데이터, 분석, 플랫폼, 스타트업, 아기, 유니콘, ...
1      [Perception, 태스크, 자동, 가공, 모델, 개발, 학습, 파이프라인, 구...
2      [멀티, 로봇, 경로, 계획, MultiAgent, Path, Finding, MA...
3      [고객, 맞춤, Copilot, Agent, 활용, 방안, 제안, Copilot, ...
4      [사내, 유저, 워크, 플로우, 분석, 개선, 공통, 문제, 해결, 획기적, 생산,...
                             ...                        
473    [빠르, 성장, 코딧, 정부, 사업, 코딧, CODIT, AI, 빅데이터, 기술, ...
474    [빠르, 성장, 코딧, BB, 서비스, 기획, 담당자, 코딧, CODIT, AI, ...
475    [빠르, 성장, 코딧, 정부, 사업, 코딧, CODIT, AI, 빅데이터, 기술, ...
476    [빠르, 성장, 코딧, BB, 콘텐츠, 마케팅, 매니저, 코딧, CODIT, AI,...
477    [빠르, 성장, 코딧, QA, 인턴, 코딧, CODIT, AI, 빅데이터, 기술, ...
Name: respons_text, Length: 478, dtype: object

In [5]:
#자격/필수 토크나이징

kiwi = Kiwi()
index = 0
df["qualifi_text"] = None
for sent in df["qualifications"]:
    sent = "" if pd.isna(sent) else str(sent)
    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)
    result = kiwi.tokenize(clean_sent)
    sub_result = []    
    for x in result:
        word = x.form
        pos = x.tag
        if pos[0] in ["J", "E", "X"] or len(word) <= 1:
            continue
        sub_result.append(word)
    print(f"index: {index}, sub_list: {sub_result}")
    if len(sub_result) > 0:
        df.at[index, "qualifi_text"] = sub_result

    index += 1


df["qualifi_text"]

index: 0, sub_list: ['Frontend', '개발', '경험', '이상', '보유', 'JavaScript', 'React', '대하', '이해', '갖추', '계시', 'Nextjs', '기반', '프로젝트', '경험', '서비스', '대하', '이해', '보유', '문제', '해결', '성능', '최적', '능력', '보이', '새롭', '기술', '빠르', '적응', '활용', '능력', '보유', '계시', '동료', '의견', '존중', '논리', '소통', '통하', '합리', '기술', '의사', '결정', '함께', '이끌', '경력', '이상', '이하', '학교', '졸업', '이상', '국문', '이력서', '경력', '기술', 'Word', '하나', '파일', '작성']
index: 1, sub_list: ['관련', '전공', '석사', '이상', '학력', '혹은', '준하', '경력', '실력', '러닝', '프로젝트', '학습', '평가', '개선', '결과물', '주도', '경험', 'vision', '또는', '멀티', '센서', '데이터', '다루', '경험', '연구', '산업', '무관', '논문', '오픈', '소스', '아이디어', '빠르', '구현', '실험', '의사', '결정', '연결', '경험']
index: 2, sub_list: ['해당', '분야', '석사', '신입', '또는', '관련', '분야', '경력', '이상', '성실', '다양', '이해관계자', '원활', '협업', '커뮤니케이션', '가능', '알고리즘', '설계', '최적', '대하', '이해', '데이터', '구조', '경로', '탐색', '알고리즘', '대하', '이해', 'JAVA', '또는', '유사', '객체', '지향', '언어', '개발', '역량', '전문', '연구', '요원', '병역', '특례', '지원', '가능']
index: 3, sub_list: ['경력', '이상', '대학', '졸업', '

0      [Frontend, 개발, 경험, 이상, 보유, JavaScript, React, ...
1      [관련, 전공, 석사, 이상, 학력, 혹은, 준하, 경력, 실력, 러닝, 프로젝트,...
2      [해당, 분야, 석사, 신입, 또는, 관련, 분야, 경력, 이상, 성실, 다양, 이...
3      [경력, 이상, 대학, 졸업, 이상, Copilot, Agent, 기획, 개발, 구...
4      [이상, 경력, 소프트웨어, 엔지니어링, FE, BE, 인프라, 배포, 모니터링, ...
                             ...                        
473    [필수, 이력서, 자기소개, 성장, 배경, 경험, 업무, 대하, 작성, 정부, 기관...
474    [이렇, 합류, 필요, 레퍼런스, 체크, 진행, 필수, 이력서, 자기소개, 성장, ...
475    [필수, 이력서, 자기소개, 성장, 배경, 경험, 업무, 위주, 작문, 역량, 충분...
476    [이렇, 합류, 필요, 레퍼런스, 체크, 진행, 이상, BB, 콘텐츠, 마케팅, 경...
477                                                 None
Name: qualifi_text, Length: 478, dtype: object

In [6]:
#우대사항 토크나이징

kiwi = Kiwi()
index = 0
df["preferred_text"] = None
for sent in df["preferred_quali"]:
    sent = "" if pd.isna(sent) else str(sent)
    clean_sent = re.sub("[^a-zA-Z가-힣0-9\s]", "", sent)
    result = kiwi.tokenize(clean_sent)
    sub_result = []    
    for x in result:
        word = x.form
        pos = x.tag
        if pos[0] in ["J", "E", "X"] or len(word) <= 1:
            continue
        sub_result.append(word)
    print(f"index: {index}, sub_list: {sub_result}")
    if len(sub_result) > 0:
        df.at[index, "preferred_text"] = sub_result

    index += 1


df["preferred_text"]

index: 0, sub_list: ['TypeScript', '대하', '경험', '이해', '규모', '프로젝트', '업무', '성공', '수행', '경험', '분석', '설계', '주도', '진행', '경험', '다양', '프론트', '엔드', '프레임', '워크', '활용', '경험', '디자인', '시스템', '혹은', '컴포넌트', '라이브러리', '구축', '경험', '수준', '도메인', '지식', '필요', '서비스', '구축', '경험']
index: 1, sub_list: ['코드', '리뷰', '통하', '구조', '논의', '주도', '문제', '정의', '해결', '시키', '아니', '문제', '스스로', '파악', '방향', '제시', '모르', '영역', '탐구', '이런', '수동', '지시', '기다리', '정하', '범위', 'Autolabeling', 'weak', 'supervision', 'active', 'learning', 'uncertainty', '관련', '경험', '규모', '데이터', '처리', '실험', '추적', '버저', '경험', '자율', '주행', '도메인', '지식', '문제', '어떻', '정의', '어떤', '선택', '결과', '무엇', '배우', '드러나', '자료', '선호']
index: 2, sub_list: ['로봇', 'AI', '관련', '업무', '경험자', 'MultiAgent', 'Path', 'FindingMAPF', '알고리즘', '대하', '연구', '또는', '실무', '개발', '경험', '규모', '멀티', '로봇', '시스템', '또는', '물류', '자동', '시스템', '대하', '이해', '김주현', '이사', '대표', '컨설턴트', '헤드', '헌팅', '사업부', '토스', '파트너', '파워', 'wwwmantpcokr', '본사', '서울특별시', '마포구', '월드컵', '16', '토스', '빌딩', '지사', '경기도', '안산시', '단원

0      [TypeScript, 대하, 경험, 이해, 규모, 프로젝트, 업무, 성공, 수행,...
1      [코드, 리뷰, 통하, 구조, 논의, 주도, 문제, 정의, 해결, 시키, 아니, 문...
2      [로봇, AI, 관련, 업무, 경험자, MultiAgent, Path, Findin...
3      [Copilot, Studio, Builder, Power, Automate, Po...
4                                                   None
                             ...                        
473    [PPT, 작성, 한글, 엑셀, 문서, 작성, 자신, 새롭, 제도, 정책, 빠르, ...
474    [SaaS, 또는, IT, 솔루션, 서비스, 기획, 이상, 경험, 디자이너, 개발,...
475    [정보, 통신, AI, IT, 기술, 분야, 대하, 이해도, 관심, 정부, 기관, ...
476    [콘텐츠, 기획, 제작, 관련, 업무, 경험, 비즈니스, 특성, 이해, 잠재, 고객...
477    [모바일, 또는, 서비스, 사용, 테스트, 흥미, 반복, 테스트, 활동, 집중력, ...
Name: preferred_text, Length: 478, dtype: object

In [7]:
#채용절차/전형/지원방법/제출서류 등 토크나이징

kiwi = Kiwi()
index = 0
df["hiring_text"] = None
for sent in df["hiring_process"]:
    sent = "" if pd.isna(sent) else str(sent)
    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)
    result = kiwi.tokenize(clean_sent)
    sub_result = []    
    for x in result:
        word = x.form
        pos = x.tag
        if pos[0] in ["J", "E", "X"] or len(word) <= 1:
            continue
        sub_result.append(word)
    print(f"index: {index}, sub_list: {sub_result}")
    if len(sub_result) > 0:
        df.at[index, "hiring_text"] = sub_result

    index += 1


df["hiring_text"]

index: 0, sub_list: ['Email', 'wonmiahnapsearchcokr', '채용', '제출', '서류', '사람', '온라인', '이력서', '접수', '방법', '사람', '입사', '지원', '전형', '절차', '서류', '전형', '면접', '면접', '최종', '합격']
index: 1, sub_list: ['Spatial', 'Data', 'Processing', '합류', '과정', '서류', '검토', '코딩', '테스트', '사전', '과제', '진행', '기술', '인터뷰', '정답', '맞히', '문제', '아니']
index: 2, sub_list: ['email', 'tendermantpcokr', 'tendernavercom', '제출', '서류', '사람', '온라인', '이력서', '접수', '방법', '사람', '입사', '지원', '전형', '절차', '서류', '전형', '면접', '최종', '합격']
index: 3, sub_list: ['서류', '전형', '면접', '부문', '리더', '면접', '대표', '이사', '최종', '합격']
index: 4, sub_list: ['제출', '서류', '사람', '온라인', '이력서', '접수', '방법', '사람', '입사', '지원', '전형', '절차', '서류', '전형', '면접', '면접', '최종', '합격']
index: 5, sub_list: ['제출', '서류', '사람', '온라인', '이력서', '접수', '방법', '사람', '입사', '지원', '전형', '절차', '서류', '전형', '면접', '면접', '최종', '합격']
index: 6, sub_list: ['전형', '일정', '합격', '발표', '개별', '안내', '드리', '서류', '최종', '합격', '제출', '예정', 'leeshonycom', 'com']
index: 7, sub_list: ['제출', '서류', '자유', '양식', '이력서', '포트

0      [Email, wonmiahnapsearchcokr, 채용, 제출, 서류, 사람, ...
1      [Spatial, Data, Processing, 합류, 과정, 서류, 검토, 코딩...
2      [email, tendermantpcokr, tendernavercom, 제출, 서...
3               [서류, 전형, 면접, 부문, 리더, 면접, 대표, 이사, 최종, 합격]
4      [제출, 서류, 사람, 온라인, 이력서, 접수, 방법, 사람, 입사, 지원, 전형,...
                             ...                        
473    [지원자, 이해, 위하, 아래, 서류, 제출, 서류, 전형, 온라인, 면접, 대면,...
474    [포지션, 따르, 과제, 주어지, 지원자, 이해, 위하, 아래, 서류, 제출, ht...
475    [이상, 이하, 과제, 발굴, 수주, 관리, 정산, 사업, 과정, 대하, 경험, P...
476    [포지션, 따르, 과제, 주어지, 지원자, 이해, 위하, 아래, 서류, 제출, ht...
477    [서류, 전형, 테스트, 온라인, 면접, 대면, 면접, 최종, 합격, httpsww...
Name: hiring_text, Length: 478, dtype: object

In [8]:
#복지/근무환경/근무조건 등 토크나이징

kiwi = Kiwi()
index = 0
df["benefits_text"] = None
for sent in df["benefits"]:
    sent = "" if pd.isna(sent) else str(sent)
    clean_sent = re.sub("[^a-zA-Z가-힣0-9\s]", "", sent)
    result = kiwi.tokenize(clean_sent)
    sub_result = []    
    for x in result:
        word = x.form
        pos = x.tag
        if pos[0] in ["J", "E", "X"] or len(word) <= 1:
            continue
        sub_result.append(word)
    print(f"index: {index}, sub_list: {sub_result}")
    if len(sub_result) > 0:
        df.at[index, "benefits_text"] = sub_result

    index += 1


df["benefits_text"]

index: 0, sub_list: ['고용', '형태', '정규직']
index: 1, sub_list: ['출근', '서울', '선릉', '재택', '001600', '장비', '인프라', '개발', '장비', '지원', '금액', '제한', 'Coding', 'Agent', '구독', '금액', '제한', 'RTX', 'PRO', '6000', 'GPU', '이상', 'GPU', '자원', '예산', '확보', '성장', '주요', '학회', '참석', '학습', '기회', '장려', '초기', '멤버', '제품', '아키텍처', '함께', '설계', '본인', '경험', '역량', '드러나', '자료', '제출', '이력서', '포트폴리오', '허브', '프로젝트', '설명', '형태', '자유', '본인', '경험', '문제', '얼마나', '깊이', '고민', '아직', '경험', '영역', '어떻', '사고', '접근', '가상', '시나리오', '바탕', '문제', '정의', '해결', '방향', '논의', '형태', '진행']
index: 2, sub_list: []
index: 3, sub_list: ['365', 'CopilotAgent', '도입', '변화', '관리', '차원', '사용자', '교육', '가이드', '개발', '워크숍', '해커', '진행', '365', 'Copilot', '관련', '진단', '도입', '구축', '교육', '프로젝트', '직접', '수행', '고용', '형태', '정규직', '오전', '균형', '조화', '만들', '위하', '노력', '보험', '건강', '보험', '연금', '보험', '고용', '보험', '산재', '보험', '창립', '기념일', '휴무', '생일', '휴가', '경조', '휴가', '장기', '근속', 'Refresh', '휴가', '휴가비', '지원', '특별', '기여자', '포상', '휴가', '직원', '가족', '휴가비', '지원', '국내', '고급', '호텔',

0                                          [고용, 형태, 정규직]
1      [출근, 서울, 선릉, 재택, 001600, 장비, 인프라, 개발, 장비, 지원, ...
2                                                   None
3      [365, CopilotAgent, 도입, 변화, 관리, 차원, 사용자, 교육, 가...
4                                          [고용, 형태, 정규직]
                             ...                        
473    [정책, 자금, 융자, 창업, 기업, 운영, 비용, 부담, 위하, 정부, 차원, 저...
474    [업무, 효율, 높이, 환경, 제공, 웰컴, 키트, 최신, 노트북, 듀얼, 모니터,...
475    [업무, 효율, 높이, 환경, 제공, 웰컴, 키트, 최신, 노트북, 듀얼, 모니터,...
476    [업무, 효율, 높이, 환경, 제공, 웰컴, 키트, 최신, 노트북, 듀얼, 모니터,...
477    [업무, 효율, 높이, 환경, 제공, 웰컴, 키트, 최신, 노트북, 듀얼, 모니터,...
Name: benefits_text, Length: 478, dtype: object

In [9]:
#급여/연봉/처우 등 토크나이징

kiwi = Kiwi()
index = 0
df["salary_text"] = None
for sent in df["salary"]:
    sent = "" if pd.isna(sent) else str(sent)
    clean_sent = re.sub("[^a-zA-Z가-힣0-9\s]", "", sent)
    result = kiwi.tokenize(clean_sent)
    sub_result = []    
    for x in result:
        word = x.form
        pos = x.tag
        if pos[0] in ["J", "E", "X"] or len(word) <= 1:
            continue
        sub_result.append(word)
    print(f"index: {index}, sub_list: {sub_result}")
    if len(sub_result) > 0:
        df.at[index, "salary_text"] = sub_result

    index += 1


df["salary_text"]

index: 0, sub_list: ['이력서', '제출', '이직', '사유', '퇴직', '사유', '최종', '연봉', '희망', '연봉', '필수', '기재', '요망', '기본급', '성과급', '각각', '기재', '급여', '연봉', '추후', '협의', '급여', '면접', '결정']
index: 1, sub_list: ['지원서', '접수', '선택', '커피챗', '기술', '인터뷰', '필요', '추가', '인터뷰', '컬처', '오프라인', '처우', '협의', '최종', '합류', '스톡옵션', '제공']
index: 2, sub_list: []
index: 3, sub_list: ['개인', '성장', '지원비', '지급', '100', '급여', '협의', '결정']
index: 4, sub_list: ['급여', '면접', '결정']
index: 5, sub_list: ['급여', '면접', '결정']
index: 6, sub_list: ['KDSTA', '소시', '실무', '능력', '향상', '교육', '근무', '조건', '고용', '형태', '정규직', '기간', '개월', '근무', '일시', '급여', '회사', '내규', '따르', '지원', '방법', '온라인', '입사', '지원', '전형', '절차', '서류', '전형', '면접', '최종', '합격', '유의', '사항', '입사', '지원', '서류', '허위', '사신', '발견', '이후', '채용', '취소']
index: 7, sub_list: ['급여', '당사', '내규', '따르', '경력', '이전', '급여', '급여', '면접', '결정', '급여', '제도', '보험']
index: 8, sub_list: ['급여', '연봉', '3000']
index: 9, sub_list: ['AI', 'Al', '기반', '서비스', '검증', '용산', '설계', '방법', '도출', '혹은', 'SIW', 'Al', '서비스', '또는', '개발

0      [이력서, 제출, 이직, 사유, 퇴직, 사유, 최종, 연봉, 희망, 연봉, 필수, ...
1      [지원서, 접수, 선택, 커피챗, 기술, 인터뷰, 필요, 추가, 인터뷰, 컬처, 오...
2                                                   None
3                     [개인, 성장, 지원비, 지급, 100, 급여, 협의, 결정]
4                                           [급여, 면접, 결정]
                             ...                        
473    [해당, 포지션, 인턴, 포지션, 개월, 협의, 가능, 야근, 단위, 수당, 지급,...
474    [서류, 전형, 온라인, 면접, 대면, 면접, 처우, 협의, 최종, 합격, 개별, ...
475    [서류, 전형, 온라인, 면접, 대면, 면접, 처우, 협의, 최종, 합격, 개별, ...
476    [서류, 전형, 온라인, 면접, 대면, 면접, 처우, 협의, 최종, 합격, 개별, ...
477    [해당, 포지션, 인턴, 포지션, 개월, 협의, 가능, 야근, 단위, 수당, 지급,...
Name: salary_text, Length: 478, dtype: object

In [10]:
df.head()

,keyword,title,company,location,career,education,work_type,deadline,link,rec_idx,...,preferred_quali,hiring_process,benefits,salary,respons_text,qualifi_text,preferred_text,hiring_text,benefits_text,salary_text
0,AI 에이전트,[AI기반 특허 데이터분석 플랫폼 기업] 프론트엔드 개발자,에이피써치,서울 강남구,경력 7~15년,대졸↑,정규직,채용시,https://www.saramin.co.kr/zf_user/jobs/relay/v...,52982351,...,TypeScript에 대한 경험과 이해가 깊은 분\n대규모 프로젝트나 복잡한 업무를...,E-mail : wonmi.ahn@apsearch.co.kr\n===========...,고용형태 : 정규직,"이력서 제출시 이직사유(퇴직사유), 최종연봉 및 희망연봉 필수 기재요망(기본급/성과...","[회사, AI, 기반, 특허, 데이터, 분석, 플랫폼, 스타트업, 아기, 유니콘, ...","[Frontend, 개발, 경험, 이상, 보유, JavaScript, React, ...","[TypeScript, 대하, 경험, 이해, 규모, 프로젝트, 업무, 성공, 수행,...","[Email, wonmiahnapsearchcokr, 채용, 제출, 서류, 사람, ...","[고용, 형태, 정규직]","[이력서, 제출, 이직, 사유, 퇴직, 사유, 최종, 연봉, 희망, 연봉, 필수, ..."
1,AI 에이전트,[자율주행] AI Engineer / 3D Auto-labeling,(주)비저너리,서울 강남구,경력무관,석사↑,정규직,채용시,https://www.saramin.co.kr/zf_user/jobs/relay/v...,52981267,...,코드 리뷰를 통해 더 좋은 구조 논의\n주도적으로 문제를 정의하고 해결하는 분\n시...,Spatial Data Processing\n합류 과정\n서류 검토\n코딩테스트나 ...,주 3일 출근 (서울 선릉) + 주 2일 재택\n00~16:00\n장비 및 인프라\...,지원서 접수 → (선택) 커피챗 → 기술 인터뷰 → 필요 시 추가 인터뷰(컬처/오프...,"[Perception, 태스크, 자동, 가공, 모델, 개발, 학습, 파이프라인, 구...","[관련, 전공, 석사, 이상, 학력, 혹은, 준하, 경력, 실력, 러닝, 프로젝트,...","[코드, 리뷰, 통하, 구조, 논의, 주도, 문제, 정의, 해결, 시키, 아니, 문...","[Spatial, Data, Processing, 합류, 과정, 서류, 검토, 코딩...","[출근, 서울, 선릉, 재택, 001600, 장비, 인프라, 개발, 장비, 지원, ...","[지원서, 접수, 선택, 커피챗, 기술, 인터뷰, 필요, 추가, 인터뷰, 컬처, 오..."
2,AI 에이전트,[로봇 솔루션기업] 모바일로봇 교통 관제 시스템 설계 및 개발자,(주)맨토스파워,서울 마포구,경력2년↑,대졸↑,정규직,~ 02/17(화),https://www.saramin.co.kr/zf_user/jobs/relay/v...,52980063,...,"로봇, AI 관련 업무 유경험자\nMulti-Agent Path Finding(MA...",e-mail : tender@mantp.co.kr / tender3599@naver...,NaN,NaN,"[멀티, 로봇, 경로, 계획, MultiAgent, Path, Finding, MA...","[해당, 분야, 석사, 신입, 또는, 관련, 분야, 경력, 이상, 성실, 다양, 이...","[로봇, AI, 관련, 업무, 경험자, MultiAgent, Path, Findin...","[email, tendermantpcokr, tendernavercom, 제출, 서...",None,None
3,AI 에이전트,Microsoft 365 Copilot 및 Agent 개발 컨설턴트,(주)무늬랩스,서울 송파구,경력2년↑,초대졸↑,정규직,~ 03/04(수),https://www.saramin.co.kr/zf_user/jobs/relay/v...,52979929,...,"Copilot Studio (Builder), Power Automate, Powe...",서류전형 → 1차면접 (부문 리더) → 2차면접 (대표이사) → 최종합격,M365/Copilot/Agent 도입 변화관리 차원의 사용자 교육 및 가이드 개발...,개인성장지원비 지급 (인당 연 100만원\n급여: 협의 후 결정,"[고객, 맞춤, Copilot, Agent, 활용, 방안, 제안, Copilot, ...","[경력, 이상, 대학, 졸업, 이상, Copilot, Agent, 기획, 개발, 구...","[Copilot, Studio, Builder, Power, Automate, Po...","[서류, 전형, 면접, 부문, 리더, 면접, 대표, 이사, 최종, 합격]","[365, CopilotAgent, 도입, 변화, 관리, 차원, 사용자, 교육, 가...","[개인, 성장, 지원비, 지급, 100, 급여, 협의, 결정]"
4,AI 에이전트,35. AI Platform Engineer [대기업 게임사],베스트네트워크㈜,서울 강남구,경력2년↑,대졸↑,정규직,~ 02/17(화),https://www.saramin.co.kr/zf_user/jobs/relay/v...,52979323,...,NaN,제출서류: 사람인 온라인 이력서\n접수방법: 사람인 입사지원\n전형절차: 서류전형 ...,고용형태 : 정규직,급여: 면접 후 결정,"[사내, 유저, 워크, 플로우, 분석, 개선, 공통, 문제, 해결, 획기적, 생산,...","[이상, 경력, 소프트웨어, 엔지니어링, FE, BE, 인프라, 배포, 모니터링, ...",None,"[제출, 서류, 사람, 온라인, 이력서, 접수, 방법, 사람, 입사, 지원, 전형,...","[고용, 형태, 정규직]","[급여, 면접, 결정]"


In [11]:
# AI 에이전트와 관련없는 값 삭제하기
del_data = [52923369 , 52895376, 52884267, 52884229, 52884099, 52833237, 52731483, 52731459, 52651246]

df = df[~df["rec_idx"].isin([52923369 , 52895376, 52884267, 52884229, 52884099, 52833237, 52731483, 52731459, 52651246])]





In [ ]:

def make_career_group(val):
    t = "" if pd.isna(val) else str(val).strip()
    t = re.sub(r"\s+", " ", t)

    if not t:
        return "미상/기타"

    # 경력무관
    if "무관" in t:
        return "경력무관"

    # 신입 / 신입·경력
    if "신입" in t and "경력" in t:
        return "신입/경력"
    if "신입" in t and "경력" not in t:
        return "신입"

    # 범위: "7~15년"
    m = re.search(r"(\d+)\s*~\s*(\d+)\s*년", t)
    if m:
        mn = int(m.group(1))
        # 그룹은 min 기준(원하면 max 기준/중간값 기준도 가능)
        if mn <= 3:  return "1-3년"
        if mn <= 6:  return "4-6년"
        if mn <= 10: return "7-10년"
        return "11년+"

    # 이상/↑: "경력2년↑", "2년 이상"
    m = re.search(r"(\d+)\s*년\s*(?:↑|이상)", t)
    if m:
        mn = int(m.group(1))
        if mn <= 3:  return "1-3년"
        if mn <= 6:  return "4-6년"
        if mn <= 10: return "7-10년"
        return "11년+"

    # "경력 3년" 같은 형태
    m = re.search(r"경력\s*(\d+)\s*년", t)
    if m:
        mn = int(m.group(1))
        if mn <= 3:  return "1-3년"
        if mn <= 6:  return "4-6년"
        if mn <= 10: return "7-10년"
        return "11년+"

    return "미상/기타"

# ✅ 그룹 컬럼 생성
df["career_group"] = df["career"].apply(make_career_group)


In [12]:
df.info()

<class 'pandas.DataFrame'>
Index: 469 entries, 0 to 477
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   keyword           469 non-null    str   
 1   title             469 non-null    str   
 2   company           469 non-null    str   
 3   location          469 non-null    str   
 4   career            469 non-null    str   
 5   education         469 non-null    str   
 6   work_type         469 non-null    str   
 7   deadline          469 non-null    str   
 8   link              469 non-null    str   
 9   rec_idx           469 non-null    int64 
 10  keyword_text      469 non-null    str   
 11  crawled_at        469 non-null    str   
 12  corp_domain       433 non-null    str   
 13  totaltext         469 non-null    str   
 14  responsibilities  469 non-null    str   
 15  qualifications    430 non-null    str   
 16  preferred_quali   386 non-null    str   
 17  hiring_process    442 non-null  

In [13]:
df.head()

df.to_csv(
    "../data/saramin_merge_sections_tokeni_v3.csv", 
    index=False
)